In [546]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.sparse as sp
from itertools import islice, cycle
from more_itertools import pairwise
from tqdm.auto import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
os.chdir("C:\\Users\\Администратор.WIN-U1NLG8MM702\\Documents\\Katerina\\Datasets for analysis")
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [547]:

interactions = pd.read_csv('interactions.csv')
items = pd.read_csv('items.csv')
users = pd.read_csv('users.csv')

In [758]:
forrand=pd.read_csv('interactions.csv')

In [731]:
forrand.shape

(1562617, 5)

In [733]:
forrand.shape

(323571, 5)

In [635]:
sample = pd.read_csv('sample_submission.csv') 

In [636]:
sample[sample['Id']==10309]

,Id,Predicted
50,10309,313 253 245 205 187 168 155 149 146 128 128 12...


In [550]:
interactions.shape, items.shape, users.shape

((1562617, 5), (63758, 5), (137254, 3))

### Предобработка

In [551]:
interactions['start_date'] = pd.to_datetime(interactions['start_date'])

In [552]:
duplicates = interactions.duplicated(subset=['user_id', 'item_id'], keep=False)
df_duplicates = interactions[duplicates].sort_values(by=['user_id', 'start_date'])
interactions = interactions[~duplicates]

In [553]:
df_duplicates = df_duplicates.groupby(['user_id', 'item_id']).agg({
    'progress': 'max',
    'rating': 'max',
    'start_date': 'min'
})
interactions = interactions.append(df_duplicates.reset_index(), ignore_index=True)

In [554]:
interactions['progress'] = interactions['progress'].astype(np.int8)
interactions['rating'] = interactions['rating'].astype(pd.SparseDtype(np.float32, np.nan))

In [572]:
interactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1562505 entries, 0 to 1562504
Data columns (total 5 columns):
 #   Column      Non-Null Count    Dtype               
---  ------      --------------    -----               
 0   user_id     1562505 non-null  int64               
 1   item_id     1562505 non-null  int64               
 2   progress    1562505 non-null  int8                
 3   rating      323563 non-null   Sparse[float32, nan]
 4   start_date  1562505 non-null  datetime64[ns]      
dtypes: Sparse[float32, nan](1), datetime64[ns](1), int64(2), int8(1)
memory usage: 39.7 MB


In [573]:
interactions.to_pickle('interactions_preprocessed.pickle')

In [574]:
users.head()

,user_id,age,sex
0,0,45_54,1.0
1,1,25_34,0.0
2,2,45_54,0.0
3,3,65_inf,0.0
4,4,18_24,0.0


In [575]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137254 entries, 0 to 137253
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype               
---  ------   --------------   -----               
 0   user_id  137254 non-null  int64               
 1   age      137244 non-null  category            
 2   sex      135640 non-null  Sparse[float32, nan]
dtypes: Sparse[float32, nan](1), category(1), int64(1)
memory usage: 2.2 MB


In [576]:
users.nunique()

user_id    137254
age             6
sex             2
dtype: int64

In [577]:
users['age'] = users['age'].astype('category')
users['sex'] = users['sex'].astype(pd.SparseDtype(np.float32, np.nan))


In [578]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137254 entries, 0 to 137253
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype               
---  ------   --------------   -----               
 0   user_id  137254 non-null  int64               
 1   age      137244 non-null  category            
 2   sex      135640 non-null  Sparse[float32, nan]
dtypes: Sparse[float32, nan](1), category(1), int64(1)
memory usage: 2.2 MB


In [579]:
interaction_users = interactions['user_id'].unique()

common_users = len(np.intersect1d(interaction_users, users['user_id']))
users_only_in_interaction = len(np.setdiff1d(interaction_users, users['user_id']))
users_only_features = len(np.setdiff1d(users['user_id'], interaction_users))
total_users = common_users + users_only_in_interaction + users_only_features
print(f'Кол-во пользователей - {total_users}')
print(f'Кол-во пользователей c взаимодействиями и фичами - {common_users} ({common_users / total_users * 100:.2f}%)')
print(f'Кол-во пользователей только c взаимодействиями - {users_only_in_interaction} ({users_only_in_interaction / total_users * 100:.2f}%)')
print(f'Кол-во пользователей только c фичами - {users_only_features} ({users_only_features / total_users * 100:.2f}%)')

Кол-во пользователей - 164771
Кол-во пользователей c взаимодействиями и фичами - 130808 (79.39%)
Кол-во пользователей только c взаимодействиями - 27517 (16.70%)
Кол-во пользователей только c фичами - 6446 (3.91%)


In [580]:
users.to_pickle('users_preprocessed.pickle')

In [581]:
items.head()

,id,title,genres,authors,year
0,248031,ворон-челобитчик,"Зарубежные детские книги,Сказки,Зарубежная кла...",Михаил Салтыков-Щедрин,1886
1,256084,скрипка ротшильда,"Классическая проза,Литература 19 века,Русская ...",Антон Чехов,1894
2,134166,испорченные дети,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1869
3,281311,странный человек,"Пьесы и драматургия,Литература 19 века",Михаил Лермонтов,1831
4,213473,господа ташкентцы,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1873


In [582]:
items.nunique()
#Почему колонка year типа object, а не int?

id         63758
title      57992
genres     11091
authors    17024
year        1074
dtype: int64

In [583]:
for col in ['genres', 'authors', 'year']:
    items[col] = items[col].astype('category')

In [584]:
interaction_items = interactions['item_id'].unique()

common_items = len(np.intersect1d(interaction_items, items['id']))
items_only_in_interaction = len(np.setdiff1d(interaction_items, items['id']))
items_only_features = len(np.setdiff1d(items['id'], interaction_items))
total_items = common_items + items_only_in_interaction + items_only_features
print(f'Кол-во книг - {total_items}')
print(f'Кол-во книг c взаимодействиями и фичами - {common_items} ({common_items / total_items * 100:.2f}%)')
print(f'Кол-во книг только c взаимодействиями - {items_only_in_interaction} ({items_only_in_interaction / total_items * 100:.2f}%)')
print(f'Кол-во книг только c фичами - {items_only_features} ({items_only_features / total_items * 100:.2f}%)')

Кол-во книг - 63758
Кол-во книг c взаимодействиями и фичами - 63758 (100.00%)
Кол-во книг только c взаимодействиями - 0 (0.00%)
Кол-во книг только c фичами - 0 (0.00%)


In [585]:
items.to_pickle('items_preprocessed.pickle')

In [586]:
items['title'] = items['title'].str.lower()

In [587]:
item_titles = pd.Series(items['title'].values, index=items['id']).to_dict()

In [588]:
title_items = items.groupby('title')['id'].agg(list)

### Надо заджоинить

In [589]:
items.head()

,id,title,genres,authors,year
0,248031,ворон-челобитчик,"Зарубежные детские книги,Сказки,Зарубежная кла...",Михаил Салтыков-Щедрин,1886
1,256084,скрипка ротшильда,"Классическая проза,Литература 19 века,Русская ...",Антон Чехов,1894
2,134166,испорченные дети,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1869
3,281311,странный человек,"Пьесы и драматургия,Литература 19 века",Михаил Лермонтов,1831
4,213473,господа ташкентцы,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1873


In [590]:
interactions.shape

(1562505, 5)

In [591]:
interactions.head()

,user_id,item_id,progress,rating,start_date
0,90133,82910,100,NaN,2018-01-01
1,159130,331068,70,NaN,2018-01-01
2,80061,26540,69,4.0,2018-01-01
3,12811,301895,16,NaN,2018-01-01
4,5778,127872,100,NaN,2018-01-01


In [592]:
users_id=pd.DataFrame({'user_id':users['user_id'].unique()})
users_id.shape

(137254, 1)

In [593]:
items_id=pd.DataFrame({'item_id':items['id'].unique()})
items_id.shape

(63758, 1)

In [594]:
df=interactions.copy()

In [595]:
#User Id interactions в отдельный Датафрейм
df1 = pd.DataFrame({'user_id': interactions.iloc[:,0]})
# Item Id interactions в отдельный Датафрейм
df0 = pd.DataFrame({'item_id': interactions.iloc[:,1]})

In [596]:
#ID, которые есть в users и нет в interactions
users_id1=users_id.loc[~users_id.user_id.isin(df1.user_id),]
users_id1.shape

(6446, 1)

In [597]:
#interactions=pd.concat([interactions,users_id1], ignore_index=True)

In [598]:
#ID, которые есть в items и нет в interactions
items_id1=items_id.loc[~items_id.item_id.isin(df0.item_id),]
items_id1

,item_id


In [599]:
#interactions=pd.concat([interactions,items_id1], ignore_index=True)

In [600]:
#interactions.info()

In [601]:
#interactions.head()

In [602]:
#interactions['item_id']=interactions['item_id'].fillna(-1)

In [603]:
#interactions['item_id']=interactions[pd.notnull(interactions['item_id'])].astype({"item_id": "int64"})

In [604]:
#interactions['item_id']=interactions['item_id'].astype({"item_id": "int64"})

In [605]:
#interactions['user_id']=interactions['user_id'].astype({"user_id": "int64"})

In [606]:
#interactions.info()

In [670]:
users_inv_mapping = dict(enumerate(interactions['user_id'].unique())) #последовательность уникальных юзер_айди в словаре
users_mapping = {v: k for k, v in users_inv_mapping.items()} #поменяли местами порядковый номер и айди
len(users_mapping)

158325

In [609]:
items_inv_mapping = dict(enumerate(interactions['item_id'].unique())) #последовательность уникальных айтем_айди в словаре
items_mapping = {v: k for k, v in items_inv_mapping.items()} #поменяли местами порядковый номер и айди
len(items_mapping)

63758

### Validation

In [611]:
def compute_metrics(df_true, df_pred, top_N):
    result = {}
    test_recs = df_true.set_index(['user_id', 'item_id']).join(df_pred.set_index(['user_id', 'item_id']))
    test_recs = test_recs.sort_values(by=['user_id', 'rank'])

    test_recs['users_item_count'] = test_recs.groupby(level='user_id')['rank'].transform(np.size)
    test_recs['reciprocal_rank'] = (1 / test_recs['rank']).fillna(0)
    test_recs['cumulative_rank'] = test_recs.groupby(level='user_id').cumcount() + 1
    test_recs['cumulative_rank'] = test_recs['cumulative_rank'] / test_recs['rank']
    
    users_count = test_recs.index.get_level_values('user_id').nunique()
    for k in range(1, top_N + 1):
        hit_k = f'hit@{k}'
        test_recs[hit_k] = test_recs['rank'] <= k
        result[f'Precision@{k}'] = (test_recs[hit_k] / k).sum() / users_count
        result[f'Recall@{k}'] = (test_recs[hit_k] / test_recs['users_item_count']).sum() / users_count

    result[f'MAP@{top_N}'] = (test_recs["cumulative_rank"] / test_recs["users_item_count"]).sum() / users_count
    result[f'MRR'] = test_recs.groupby(level='user_id')['reciprocal_rank'].max().mean()
    return pd.Series(result)

In [612]:
train=interactions
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1562505 entries, 0 to 1562504
Data columns (total 5 columns):
 #   Column      Non-Null Count    Dtype               
---  ------      --------------    -----               
 0   user_id     1562505 non-null  int64               
 1   item_id     1562505 non-null  int64               
 2   progress    1562505 non-null  int8                
 3   rating      323563 non-null   Sparse[float32, nan]
 4   start_date  1562505 non-null  datetime64[ns]      
dtypes: Sparse[float32, nan](1), datetime64[ns](1), int64(2), int8(1)
memory usage: 39.7 MB


In [613]:
sample.head()

,Id,Predicted
0,10001,313 253 245 205 187 168 155 149 146 128 128 12...
1,10002,313 253 245 205 187 168 155 149 146 128 128 12...
2,100152,313 253 245 205 187 168 155 149 146 128 128 12...
3,100197,313 253 245 205 187 168 155 149 146 128 128 12...
4,100284,313 253 245 205 187 168 155 149 146 128 128 12...


In [614]:
train[train.user_id==10309]

,user_id,item_id,progress,rating,start_date


In [615]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1562505 entries, 0 to 1562504
Data columns (total 5 columns):
 #   Column      Non-Null Count    Dtype               
---  ------      --------------    -----               
 0   user_id     1562505 non-null  int64               
 1   item_id     1562505 non-null  int64               
 2   progress    1562505 non-null  int8                
 3   rating      323563 non-null   Sparse[float32, nan]
 4   start_date  1562505 non-null  datetime64[ns]      
dtypes: Sparse[float32, nan](1), datetime64[ns](1), int64(2), int8(1)
memory usage: 39.7 MB


### Implicit

In [644]:
def get_coo_matrix(df, 
                   user_col='user_id', 
                   item_col='item_id', 
                   weight_col=None, 
                   users_mapping=users_mapping, 
                   items_mapping=items_mapping):
    if weight_col is None:
        weights = np.ones(len(df), dtype=np.float32)
    else:
        weights = df[weight_col].astype(np.float32)

    interaction_matrix = sp.coo_matrix((
        weights, 
        (
            df[user_col].map(users_mapping.get), 
            df[item_col].map(items_mapping.get)
        )
    ))
    return interaction_matrix

In [645]:
train_mat = get_coo_matrix(train).tocsr()
train_mat

<158325x63758 sparse matrix of type '<class 'numpy.float32'>'
	with 1046878 stored elements in Compressed Sparse Row format>

In [646]:
from implicit.nearest_neighbours import CosineRecommender, BM25Recommender, TFIDFRecommender

In [647]:
cosine_model = CosineRecommender(K=10)
cosine_model.fit(train_mat.T) #

In [648]:
top_N = 10
user_id = sample['Id'].iloc[0]
row_id = users_mapping[user_id]
print(f'Рекомендации для пользователя {user_id}, номер строки - {row_id}')

Рекомендации для пользователя 10001, номер строки - 112297


In [650]:
recs = cosine_model.recommend(row_id, train_mat, N=top_N, filter_already_liked_items=True)

In [652]:
recs = pd.DataFrame(recs, columns=['col_id', 'similarity'])
recs

,col_id,similarity
0,167,0.462178
1,30268,0.437026
2,38177,0.381475
3,25590,0.308837
4,10731,0.221652
5,44993,0.206523
6,1075,0.181818
7,30110,0.174078
8,51012,0.150756
9,60884,0.150756


In [653]:
recs['item_id'] = recs['col_id'].map(items_inv_mapping.get)
recs['title'] = recs['item_id'].map(item_titles.get)
recs

,col_id,similarity,item_id,title
0,167,0.462178,111649,после ссоры
1,30268,0.437026,296208,после – долго и счастливо
2,38177,0.381475,297168,до того как
3,25590,0.308837,186002,после
4,10731,0.221652,237354,ничего больше
5,44993,0.206523,235765,никак не меньше
6,1075,0.181818,234997,"часы, идущие назад"
7,30110,0.174078,303515,путешествие по ту сторону
8,51012,0.150756,151006,темные тени нехорошей квартиры
9,60884,0.150756,188065,скрытая реальность. книга первая. щелкунчик


In [759]:
def generate_implicit_recs_mapper(model, train_matrix, N, user_mapping, item_inv_mapping):
    def _recs_mapper(user):
        try:
            user_id = user_mapping[user]
        except KeyError:
            import random
            user_id =user_mapping[random.choice(list(forrand['user_id']))]

   
        
        #номер строки юзера в словаре user_mapping
        recs = model.recommend(user_id, 
                               train_matrix, 
                               N=N, 
                               filter_already_liked_items=True)
        return [item_inv_mapping[item] for item, _ in recs] #возвращает список айтемов из рекоменд модели при подстановки айди юзера
    return _recs_mapper

In [760]:
mapper = generate_implicit_recs_mapper(cosine_model, train_mat, top_N, users_mapping, items_inv_mapping)

In [666]:
sample=sample.rename(columns={"Id": "user_id", "Predicted": "Predicted"})

In [761]:
%time
recs = pd.DataFrame({
    'user_id': sample['user_id'].unique()
})
recs['item_id'] = recs['user_id'].map(mapper)
recs.head()

Wall time: 0 ns


,user_id,item_id
0,10001,"[111649, 296208, 297168, 186002, 237354, 23576..."
1,10002,"[240408, 146455, 216971, 34530, 173125, 390138..."
2,100152,"[254229, 167562, 164132, 103839, 287084, 20505..."
3,100197,"[155301, 28353, 384765, 154214, 231681, 145823..."
4,100284,"[240559, 168733, 170741, 21375, 312572, 164263..."


In [750]:
recs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3074 entries, 0 to 3073
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  3074 non-null   int64 
 1   item_id  3074 non-null   object
dtypes: int64(1), object(1)
memory usage: 48.2+ KB


In [751]:
#Transform each element of a list-like to a row, replicating index values.
recs = recs.explode('item_id')
recs['rank'] = recs.groupby('user_id').cumcount() + 1

In [770]:
recs

,user_id,item_id
0,10001,"[111649, 296208, 297168, 186002, 237354, 23576..."
1,10002,"[240408, 146455, 216971, 34530, 173125, 390138..."
2,100152,"[254229, 167562, 164132, 103839, 287084, 20505..."
3,100197,"[155301, 28353, 384765, 154214, 231681, 145823..."
4,100284,"[240559, 168733, 170741, 21375, 312572, 164263..."
...,...,...
3069,99734,"[213245, 106590, 216658, 45238, 260428, 377851..."
3070,99757,"[286258, 234893, 181195, 267855, 386293, 31250..."
3071,99772,"[330775, 223652, 3869, 315239, 230718, 262718,..."
3072,99827,"[149928, 106113, 128133, 316428, 280472, 24774..."


In [768]:
sample.explode('Predicted')

,user_id,Predicted
0,10001,313 253 245 205 187 168 155 149 146 128 128 12...
1,10002,313 253 245 205 187 168 155 149 146 128 128 12...
2,100152,313 253 245 205 187 168 155 149 146 128 128 12...
3,100197,313 253 245 205 187 168 155 149 146 128 128 12...
4,100284,313 253 245 205 187 168 155 149 146 128 128 12...
...,...,...
3069,99734,313 253 245 205 187 168 155 149 146 128 128 12...
3070,99757,313 253 245 205 187 168 155 149 146 128 128 12...
3071,99772,313 253 245 205 187 168 155 149 146 128 128 12...
3072,99827,313 253 245 205 187 168 155 149 146 128 128 12...


In [769]:
sample

,user_id,Predicted
0,10001,313 253 245 205 187 168 155 149 146 128 128 12...
1,10002,313 253 245 205 187 168 155 149 146 128 128 12...
2,100152,313 253 245 205 187 168 155 149 146 128 128 12...
3,100197,313 253 245 205 187 168 155 149 146 128 128 12...
4,100284,313 253 245 205 187 168 155 149 146 128 128 12...
...,...,...
3069,99734,313 253 245 205 187 168 155 149 146 128 128 12...
3070,99757,313 253 245 205 187 168 155 149 146 128 128 12...
3071,99772,313 253 245 205 187 168 155 149 146 128 128 12...
3072,99827,313 253 245 205 187 168 155 149 146 128 128 12...


In [764]:
metrics = compute_metrics(sample, recs.explode('item_id'), top_N)
metrics

KeyError: "None of ['item_id'] are in the columns"

### Функция для конвертации датафрейма с рекомендациями в нужный формат для сабмита

In [753]:
def make_submission(df, id_col='user_id', predicted_col='item_id'):
    
    df_copy = df.copy()
    df_copy.loc[:, id_col] = df_copy[id_col].astype(str)
    df_copy.loc[:, predicted_col] = df_copy[predicted_col].astype(str)
    df_copy = df_copy.groupby(id_col, as_index=False).agg({predicted_col: lambda x: ' '.join(list(x))})
    df_copy.rename(columns={id_col: 'Id', predicted_col: 'Predicted'}, inplace=True)
    
    return df_copy[['Id', 'Predicted']]

In [754]:
submission_recs = make_submission(recs)
submission_recs.head()

,Id,Predicted
0,10001,111649 296208 297168 186002 237354 235765 2349...
1,10002,240408 146455 216971 34530 173125 390138 15014...
2,100152,254229 167562 164132 103839 287084 205056 3486...
3,100197,155301 28353 384765 154214 231681 145823 14317...
4,100284,240559 168733 170741 21375 312572 164263 27372...


In [755]:
submission_recs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3074 entries, 0 to 3073
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Id         3074 non-null   object
 1   Predicted  3074 non-null   object
dtypes: object(2)
memory usage: 72.0+ KB


In [756]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3074 entries, 0 to 3073
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    3074 non-null   int64 
 1   Predicted  3074 non-null   object
dtypes: int64(1), object(1)
memory usage: 48.2+ KB


In [757]:
submission_recs.to_csv('submission_recs2.csv', index=False)